### Researched Quantities

- Building this model based on a 10-year rainstorm, which is 5.5 inches in 24 hours.
-Here is the link to the sheet with the data about the plants https://1drv.ms/x/c/348b84fad346e9f1/EX-eKXt_eg1HnNybvuDh2aUBIBjV-u4fKmu33ILq_tpZVw?e=BLTTLG 
- Unit conversions:
    - 1 square mile = 640 acres
    - 1 square mile = 27,878,400 square feet
    - 1 foot = 12 inches
    - 1 gallon = 0.13368 cubic feet
- According to HomeAdvisor, it costs between $50 to $250 per linear foot to insall a sewer line. I feel like since it's NYC it'll be more expensive but it's so much that they can probably get pretty good pricing so I'm just going to guess $150 per linear foot.
    https://www.homeadvisor.com/cost/plumbing/install-a-sewer-main/
- according to NYC waterboards, the sewer rate $7.14 / ft^3 = $954479.17 $/ MG in 2024: https://www.nyc.gov/site/nycwaterboard/rates/rates-regulations.page

In [24]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables
Pkg.add("XLSX")

  Activating project at `c:\Users\grcra\OneDrive\Desktop\FA24\BEE4750\4750project`
   Installed ZipFile ──────── v0.10.1
   Installed InputBuffers ─── v1.0.0
   Installed CodecInflate64 ─ v0.1.3
   Installed EzXML ────────── v1.2.0
   Installed ZipArchives ──── v2.4.0
   Installed XLSX ─────────── v0.10.4
Precompiling project...
  ✓ CodecInflate64
  ✓ InputBuffers
  ✓ ZipFile
  ✓ EzXML
  ✓ ZipArchives
  ✓ XLSX
  6 dependencies successfully precompiled in 52 seconds. 233 already precompiled.
    Updating registry at `C:\Users\grcra\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\grcra\OneDrive\Desktop\FA24\BEE4750\4750project\Project.toml`
  No Changes to `C:\Users\grcra\OneDrive\Desktop\FA24\BEE4750\4750project\Manifest.toml`


In [337]:
#deriving objective function
import XLSX
data = XLSX.readxlsx("WWTP_data.xlsx")

facilities = data["Sheet1!A2:A15"]
design_capacity = data["Sheet1!B2:B15"]
population_served = data["Sheet1!C2:C15"]
sanitary_demand = 80*(10^-6)*population_served #MGD = (gal/(day* person)) * people * 10^-6 MGD/(gal/d)


cost_per_mile = 150*5280 #$/ft * ft/mile
distance = data["Sheet1!E2:E15"]
W_coef = distance*cost_per_mile

# storm_height = 5.5/12 #feet / day for 10 year storm
storm_height = 0.5/12 #feet / day example for below capacity (I played with this number a bit)

drainage_area = data["Sheet1!D2:D15"] #acres = (miles^2)/640
storm_vol = (storm_height * drainage_area)/3.06888 #MGD = ((in/day)*acre)/(3.06888 acre foot / Mgal)
treatment_cost = 954479.17 #dollars / MGD
CSO_cost = 954490; #dollars/Mgal (I've been playing around with this number)

In [74]:
#OLD (wanted to keep it here in case I messed things up) #Setting Up Optimization
# stormwater_model = Model(HiGHS.Optimizer) #initialize model object

# #create variables: amount of wastewater diverted to new plant
# @variable(stormwater_model, W[1:14], Bin) #will we divert water to the new plant?
# @variable(stormwater_model, Q[1:14] >= 0) #how much water will go to the new plant
# @variable(stormwater_model, O[1:14], Bin) #will we get an overflow in a 10-year storm

# #create objective function
# @objective(stormwater_model, Min, sum(W_coef .* W) + sum(treatment_cost .* Q) + sum(CSO_cost .* O))

# #constraints
# for i in 1:14
#     @constraint(stormwater_model, O[i] => {storm_vol[i] .+ sanitary_demand[i] <= design_capacity[i]})
# end

In [ ]:
#Setting Up Optimization
stormwater_model = Model(HiGHS.Optimizer) #initialize model object

#create variables: does each plant divert stormwater  to new plant or allow a CSO
@variable(stormwater_model, W[1:14], Bin) #are we over capacity?
@variable(stormwater_model, Q[1:14] >= 0) #how much water will go to the new plant
@variable(stormwater_model, O[1:14] >= 0) #how much will CSO

#create objective function
@objective(stormwater_model, Min, sum(W_coef .* W) + sum(treatment_cost .* Q) + sum(CSO_cost .* O))

#constraints
for i in 1:14
    @constraint(stormwater_model, !W[i] => {storm_vol[i] + sanitary_demand[i] <= design_capacity[i]})
    @constraint(stormwater_model, Q[i] + O[i] == W[i]*(storm_vol[i] + sanitary_demand[i] - design_capacity[i]))
end

In [ ]:
#optimize
optimize!(stormwater_model)
@show value.(W)
@show value.(Q)
@show value.(O)
# @show value.(storm_vol)
# @show value.(design_capacity)
@show objective_value(stormwater_model);

Coefficient ranges:
  Matrix [1e+00, 3e+02]
  Cost   [1e+06, 1e+07]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 3e+02]
Presolving model
10 rows, 10 cols, 10 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   1251538402.101  1251538402.101     0.00%        0      0      0         0     0.1s

Solving report
  Status            Optimal
  Primal bound      1251538402.1
  Dual bound        1251538402.1
  Gap               0% (tol

In [344]:
W = value.(W)
Q = value.(Q)
O = value.(O)

# Calculate the contribution to the objective function for each plant (index)
cost_per_plant = []
for i in 1:14
    contrib = W_coef[i] * W[i] + treatment_cost * Q[i] + CSO_cost * O[i]
    push!(cost_per_plant, contrib)
end
    
storm_vol = vec(storm_vol)
plant_name = vec(facilities)
design_cap = vec(design_capacity)
sanitary_demand = vec(sanitary_demand)
table = DataFrame(Plant_Name = plant_name, Design_Capacity=design_cap, Sanitary_Demand=sanitary_demand, Storm_Vol=storm_vol, Over_Capacity=Bool.(W), Diverted_Flow=Q, CSO=O, Cost=cost_per_plant)
@show table;


table = 14×8 DataFrame
 Row │ Plant_Name           Design_Capacity  Sanitary_Demand  Storm_Vol  Over_Capacity  Diverted_Flow  CSO      Cost
     │ Any                  Any              Float64          Float64    Bool           Float64        Float64  Any
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ 26th Ward WRRF       85                      22.6742     80.2003           true        17.8745      0.0  2.51788e7
   2 │ Bowery Bay WRRF      150                     67.8662    206.414            true       124.28        0.0  1.26186e8
   3 │ Coney Island WRRF    110                     47.7061    204.839            true       142.545       0.0  1.43976e8
   4 │ Hunts Point WRRF     200                     54.7655    226.25             true        81.0153      0.0  8.60394e7
   5 │ Jamaica WRRF         100                     58.2498    343.679            true       301.928       0.0  2.9848e8
   6 │ Newto